# 영화(드라마) 추천 시스템 with PyQt
- 데이터셋은 Mydramalist 에서 약 1700개 드라마에 대한 1만개 리뷰
- 리뷰들을 TF-IDF 벡터화를 통해 단어에 대한 중요도를 설정합니다.
- 사용자 입력에 TF-IDF 벡터를 생성하고 이미 TF-IDF 벡터화된 리뷰 데이터와 코사인 유사도를 계산합니다.
- 사용자 입력과 유사한 리뷰를 찾고 알려줍니다.

In [ ]:
import sys
import os
import re
import PyQt5.QtWidgets as QtWidgets
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QTextEdit, QLabel, QListView, QVBoxLayout, QWidget, QCheckBox
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import QStringListModel

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# import requests
# from bs4 import BeautifulSoup
# from user_agent import generate_user_agent # 추가 해야 크롤링 되네!
# from urllib.request import urlopen, Request
# from PyQt5.QtWebEngineWidgets import QWebEngineView # 추가: 웹 엔진 위젯 임포트
# from PyQt5.QtCore import QUrl

# 파일 불러오기
reviews_df = pd.read_csv("../data/reviews.csv")
reviews_df['review_text'].fillna('', inplace=True)

# VADER 감정 분석기 초기화
analyzer = SentimentIntensityAnalyzer()

def sentiment_label(text):
    sentiment = analyzer.polarity_scores(text)
    if sentiment['compound'] >= 0.05:
        return 'Positive'
    elif sentiment['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
# 여기에서 reviews_df를 적절히 설정해야 합니다.
tfidf_matrix = tfidf_vectorizer.fit_transform(reviews_df['review_text'])

# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 추천 함수 정의
def get_recommendations(user_input, num_recommendations=5, similarity_threshold=0.1):
    # 사용자 입력에 대한 TF-IDF 벡터 생성
    user_input_tfidf = tfidf_vectorizer.transform([user_input])

    # 코사인 유사도 계산
    cosine_sim_user = linear_kernel(user_input_tfidf, tfidf_matrix)

    # 유사한 드라마 인덱스 찾기
    similar_indices = cosine_sim_user[0].argsort()[:-num_recommendations-1:-1]

    # 추천 드라마 정보 초기화
    recommended_movies = []

    # 추천 드라마 정보 채우기
    for idx in similar_indices:
        similarity_score = cosine_sim_user[0][idx]

        # 유사도 점수가 기준보다 큰 경우에만 추가
        if similarity_score >= similarity_threshold:
            title = reviews_df['title'].iloc[idx]
            overall_score = reviews_df['overall_score'].iloc[idx]

            # Sentiment 분석
            review_text = reviews_df['review_text'].iloc[idx]
            sentiment = sentiment_label(review_text)

            recommended_movies.append({
                'title': title,
                'overall_score': overall_score,
                'Sentiment': sentiment,
                'review_text': review_text  # review_text 정보 추가
            })

    return recommended_movies

class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setGeometry(100, 100, 1000, 800)
        self.setWindowTitle('드라마 추천 애플리케이션')

        # A 버튼 생성
        self.btn_A = QPushButton('드라마 추천', self)
        self.btn_A.setGeometry(50, 30, 100, 50)
        self.btn_A.clicked.connect(self.btn_A_clicked)

        # B 버튼 생성
        self.btn_B = QPushButton('조홍기', self)
        self.btn_B.setGeometry(50, 520, 100, 50)
        self.btn_B.clicked.connect(self.btn_B_clicked)

        # C 버튼 생성
        self.btn_C = QPushButton('문서영', self)
        self.btn_C.setGeometry(170, 520, 100, 50)
        self.btn_C.clicked.connect(self.btn_C_clicked)

        # D 버튼 생성
        self.btn_D = QPushButton('한승준', self)
        self.btn_D.setGeometry(290, 520, 100, 50)
        self.btn_D.clicked.connect(self.btn_D_clicked)

        # E 버튼 생성
        self.btn_E = QPushButton('넷플릭스', self)
        self.btn_E.setGeometry(500, 520, 100, 50)
        self.btn_E.clicked.connect(self.btn_E_clicked)
        self.label_E = QLabel('키노라이츠 인기도 기준으로 선정한 OTT별 랭킹 차트\n\n https://m.kinolights.com/', self)  # 넷플릭스 버튼 아래에 글씨 추가
        self.label_E.setGeometry(500, 580, 400, 60)

        # F 버튼 생성
        self.btn_F = QPushButton('쿠팡플레이', self)
        self.btn_F.setGeometry(620, 520, 100, 50)
        self.btn_F.clicked.connect(self.btn_F_clicked)

        # G 버튼 생성
        self.btn_G = QPushButton('디즈니+', self)
        self.btn_G.setGeometry(740, 520, 100, 50)
        self.btn_G.clicked.connect(self.btn_G_clicked)

        # H 버튼 생성
        self.btn_H = QPushButton('박스오피스', self)
        self.btn_H.setGeometry(860, 520, 100, 50)
        self.btn_H.clicked.connect(self.btn_H_clicked)

        # 결과를 표시할 QTextEdit 위젯 생성
        self.result_text = QTextEdit(self)
        self.result_text.setGeometry(50, 90, 700, 400)
        self.result_text.setReadOnly(True)

        # ListView와 모델 생성
        self.list_view = QListView(self)
        self.list_model = QStringListModel()

        # 데이터 생성 및 수정된 단어로 초기화
        self.data = []  # 초기에는 빈 리스트
        self.list_model.setStringList(self.data)

        self.list_view.setModel(self.list_model)
        self.list_view.setGeometry(50, 580, 400, 200)
        self.list_view.clicked.connect(self.list_item_clicked)
        self.list_view.hide()  # 초기에 숨김

        # 이미지를 표시할 QLabel 생성
        self.image_label = QLabel(self)
        self.image_label.setGeometry(50, 90, 700, 400)
        self.image_label.hide()  # 초기에 숨김

    def btn_A_clicked(self):
        self.image_label.hide()
        self.list_view.hide()

        user_input, ok = QtWidgets.QInputDialog.getText(self, '드라마 추천', '드라마 추천을 위한 단어를 입력하세요:')
        if ok:
            recommendations = get_recommendations(user_input)
            result_str = f"{user_input}와(과) 관련된 유사한 드라마 {len(recommendations)}편을 추천합니다:\n"
            for idx, movie in enumerate(recommendations, start=1):
                result_str += f"{idx}. 드라마 제목 [ {movie['title']} ] 평점: {movie['overall_score']}, Sentiment: {movie['Sentiment']}\n\n"
                result_str += f" 관련 드라마 리뷰: {movie['review_text']}\n"
                result_str += "- " * 80 + "\n"
            self.result_text.setPlainText(result_str)

    def btn_B_clicked(self):
        self.update_list_view([])  # 빈 리스트로 업데이트
        self.result_text.clear()
        self.image_label.clear()
        self.list_view.hide()
        self.image_label.hide()

        data_B = ["1", "2", "3"]
        self.update_list_view(data_B)

    def btn_C_clicked(self):
        self.image_label.clear()
        data_C = ["4", "5", "6"]
        self.update_list_view(data_C)

    def btn_D_clicked(self):
        self.image_label.clear()
        data_D = ["3.Amount of Kdramas_2015_2023", "3.Distribution of Overall Scores", "3.Drama_name_wordcloud", "3.Synopsis_wordcloud"]
        self.update_list_view(data_D)

    def btn_E_clicked(self):
        self.image_label.clear()
        data_E = ["9월_2주차_TOP5","9월_1주차_TOP5","8월_4주차_TOP5","8월_3주차_TOP5","8월_2주차_TOP5","8월_1주차_TOP5"]
        self.update_list_view(data_E)

    def btn_F_clicked(self):
        self.image_label.clear()
        data_F = ["9월_2주차_TOP5","9월_1주차_TOP5","8월_4주차_TOP5","8월_3주차_TOP5","8월_2주차_TOP5","8월_1주차_TOP5"]
        self.update_list_view(data_F)

    def btn_G_clicked(self):
        self.image_label.clear()
        data_G = ["9월_2주차_TOP5","9월_1주차_TOP5","8월_4주차_TOP5","8월_3주차_TOP5","8월_2주차_TOP5","8월_1주차_TOP5"]
        self.update_list_view(data_G)

    def btn_H_clicked(self):
        self.image_label.clear()
        data_H = ["9월_2주차_TOP5","9월_1주차_TOP5","8월_4주차_TOP5","8월_3주차_TOP5","8월_2주차_TOP5","8월_1주차_TOP5"]
        self.update_list_view(data_H)

    def update_list_view(self, data):
        self.data = data
        self.list_model.setStringList(self.data)
        self.list_view.show()

    def list_item_clicked(self, index):
        selected_item = index.data()
        image_path = f'../data/{selected_item}.png'  # 이미지 경로 생성
        if os.path.exists(image_path):
            pixmap = QPixmap(image_path)
            self.image_label.setPixmap(pixmap)
            self.image_label.setScaledContents(True)
            self.image_label.show()
        else:
            self.image_label.clear()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    mywindow = MyWindow()
    mywindow.show()
    app.exec_()